# Imports

In [2]:
import pandas as pd
import numpy as np
import data_processing_pipeline

#need to import a scoring metric here
from sktime.performance_metrics.forecasting import mean_absolute_scaled_error

# Functions

# Data + Pre-Processing

## Buildiers

## Scoring

Mean Anbolute Scaled Error (MASE) is a scoring metric good for our timeseries data and forecasting approaches as it ratios the Mean Absolute Error (MAE) of a model against its naively assumed MAE (called this $\text{MAE}_\alpha$). Thus the resulting equation for MASE is as follows:

$$
\text{MASE} = \frac{\text{MAE}}{\text{MAE}_\alpha}
$$

We focus first on a naive MAE which is computed based on naive forecasting. Naive forecasting is is a simplistic approach towards forecasting which makes the assumption that a future value remain mostly unchanged from the last previous value - ergo it does not take into account exogenous conditions and featuress that may influence the trend of a value over time. Naive forecasting is done often for establishing a benchmark or baseline.

For the purpose of this project, we are going to construct our own function that we can use for measuring the MASE of our predictions. 

We start with showcasing the formula for the naive MAE, $\text{MAE}_\alpha$. This specific formula looks over an entire duration of timesteps...

$$
\text{MAE}_\alpha = \frac{1}{N-1} \sum_{i = 2}^{N} \text{abs}(y_i - y_{i-1})
$$

Now if we incorporate a seasonal variation from the forecast, we would then equate the forecast to an actual value from the period, but from the previous season corresponding to the current period (i.e. this year vs last year)...

Thus we have:

$$
\text{MAE}_\alpha = \frac{1}{N-m} \sum_{i = m+1}^{N} \text{abs}(y_i - y_{i-m})
$$

Above we cover the MAE for the naive forecast. To continue building our MASE metric, we would also leverage an MAE on our new algorithm meant to construct predictions (call then $o_n$$)

Therefore:

$$
\text{MAE} = \frac{1}{N}\sum_{j=1}^{N} \text{abs}(o_i - y_i)
$$


Finally, the MASE is then constructured from these error metrics as:


$$
\text{MASE} = \frac{\text{MAE}}{\text{MAE}_\alpha}
$$

As a ratio, MASE is measured around the value of 1. When the MAE from our model performs better than the MAE of the naive assumption, the MASE ratio goes below the value of 1. Above the value of one indicates our model's error is much stronger than the naive error, indicating the model performs very poorly. 


Sources: https://medium.com/@ashishdce/mean-absolute-scaled-error-mase-in-forecasting-8f3aecc21968

Sources: https://www.launchfulfillment.com/inventory-management-naive-forecasting/

# SARIMAX Re-attempt

In [ ]:
base_df = data_processing_pipeline.run_base_df()

base_df.head()

## Old Problem and Re-execution

## Model Packaging and Deployment onto Other Categories